In [1]:
import numpy as np
import tonic
import tonic.transforms as transforms
import pandas as pd
from torch.utils.data import Dataset, DataLoader
#from tonic.dataset import Dataset
from typing import Callable, Optional
import torch
import matplotlib.pyplot as plt
from torchvision.ops import masks_to_boxes
from torchvision.utils import draw_bounding_boxes
import matplotlib.patches as patches
from tqdm.notebook import tqdm

In [2]:
import sys
sys.path.append("./mnist_sg_cnn")

In [3]:
import utils
import snn_utils
import base_model
import lenet_decolle_model

In [4]:
[torch.cuda.device(i) for i in range(torch.cuda.device_count())]

In [5]:
dir = "/media/user/EVIMO/raw/imo/eval/scene15_dyn_test_01/left_camera/ground_truth_000000"
sensor_size = [640, 480, 2]
batch_size = 6
num_bins_per_frame = 4
test_num_bins_per_frame = 8
framerate = 200
device = 'cuda'
epochs=20

In [6]:
class EVIMO(Dataset):
    def __init__(self,
                 dir: str,
                 item_to_find: int,
                 num_bins_per_frame: int,
                 #start_idx: int,
                ):
        self.dir = dir
        self.item_to_find = item_to_find
        self.num_bins_per_frame = num_bins_per_frame
        self.length = np.load(self.dir + "/length.npy")
        #print(self.length)
        #self.start_idx = start_idx


    def __getitem__(self, index):
        item = np.load(self.dir + "/" + str(index) + ".npy", allow_pickle=True).tolist()
        if self.item_to_find in item["objs_in_mask"]:
            target = np.asarray([0, 1])
        else:
            target = np.asarray([1, 0])

        target = np.tile(target, (self.num_bins_per_frame, 1))

        return {"data": torch.from_numpy(item["binned_events"]), "target": torch.from_numpy(target)}

    def __len__(self) -> int:
        return self.length # - self.start_idx
    
    

In [7]:
class TestDataset(Dataset):
    def __getitem__(self, index):
        if index < 512:
            return {"data": torch.zeros([test_num_bins_per_frame, 2, 480, 640]), "target": np.tile(torch.from_numpy(np.array([1, 0])), (test_num_bins_per_frame,1))}
        else:
            return {"data": torch.ones([test_num_bins_per_frame, 2, 480, 640]), "target": np.tile(torch.from_numpy(np.array([0, 1])), (test_num_bins_per_frame,1))}

    def __len__(self) -> int:
        return 1024

In [8]:
test_dataset = TestDataset()
tonic_dataset = EVIMO(dir="./data/EVIMO/left_cam/scene13_test5", item_to_find=23, num_bins_per_frame=num_bins_per_frame)

In [9]:
num_ones = 0
targ_set = set()

for i in range(0, len(tonic_dataset)):
    item = tonic_dataset[i]
    answers = np.unique(item["target"].argmax()).tolist()
    for a in answers:
        if a == 1:
            num_ones += 1
        targ_set.add(a)
    

In [10]:
targ_set

{0, 1}

In [11]:
num_ones / len(tonic_dataset)

0.5031446540880503

In [12]:
batched_trainloader = DataLoader(tonic_dataset, batch_size=batch_size, shuffle=False) # collate_fn=tonic.collation.PadTensors(),
test_trainloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [13]:
#for i_batch, sample_batched in enumerate(trainloader):
#    print(i_batch, sample_batched['data'].size(), sample_batched['target'].size())

In [14]:
datait = iter(batched_trainloader)
batched_data = next(datait)

In [15]:
batched_data['data'].shape

torch.Size([6, 4, 2, 480, 640])

In [16]:
batched_data['target'].transpose(0, 1)

tensor([[[0, 1],
         [0, 1],
         [0, 1],
         [0, 1],
         [0, 1],
         [0, 1]],

        [[0, 1],
         [0, 1],
         [0, 1],
         [0, 1],
         [0, 1],
         [0, 1]],

        [[0, 1],
         [0, 1],
         [0, 1],
         [0, 1],
         [0, 1],
         [0, 1]],

        [[0, 1],
         [0, 1],
         [0, 1],
         [0, 1],
         [0, 1],
         [0, 1]]])

In [17]:
batched_data['target'].shape

torch.Size([6, 4, 2])

In [18]:
batched_data['target']

tensor([[[0, 1],
         [0, 1],
         [0, 1],
         [0, 1]],

        [[0, 1],
         [0, 1],
         [0, 1],
         [0, 1]],

        [[0, 1],
         [0, 1],
         [0, 1],
         [0, 1]],

        [[0, 1],
         [0, 1],
         [0, 1],
         [0, 1]],

        [[0, 1],
         [0, 1],
         [0, 1],
         [0, 1]],

        [[0, 1],
         [0, 1],
         [0, 1],
         [0, 1]]])

In [19]:
datait = iter(test_trainloader)
batched_data = next(datait)

In [20]:
batched_data['data'].shape

torch.Size([6, 8, 2, 480, 640])

In [21]:
item = next(iter(batched_trainloader))
data = item['data']
target = item['target']

loss = torch.nn.SmoothL1Loss()

def decolle_loss(r, s, tgt):
    loss_tv = 0
    for i in range(len(r)):
        if r[i].shape != tgt.shape:
            print(f"Loss Readout shape : {r[i].shape}")
            print(f"Loss Target shape : {tgt.shape}")
        loss_tv += loss(r[i],tgt) 
    return loss_tv


convnet_sg = lenet_decolle_model.LenetDECOLLE(out_channels=2,
                    Nhid=[128, 256], #Number of convolution channels
                    Mhid=[64],
                    kernel_size=[8, 16],
                    pool_size=[4, 8],
                    input_shape=[2, 480, 640],  # data.shape[1:],
                    alpha=[.95],
                    alpharp=[.65],
                    beta=[.92],
                    num_conv_layers=2,
                    num_mlp_layers=1,
                    lc_ampl=0.5).to(device)

#net = lenet_decolle_model.LenetDECOLLE(Nhid=[1,8],Mhid=[32,64],out_channels=2, input_shape=[2, 480, 640]).to(device)

convnet_sg

print(data.shape)

data_d = data.to(device)
target_d = target.to(device)
convnet_sg.init_parameters(data_d) # Modifies readout dim

LAYER SIZE: 2421888
STDV: 0.00032128686926965593
LAYER SIZE: 68096
STDV: 0.0019160604358905182
LAYER SIZE: 64
STDV: 0.0625
torch.Size([6, 4, 2, 480, 640])


In [22]:
data_d.shape

torch.Size([6, 4, 2, 480, 640])

In [23]:
#data_d.transpose(0, 1).shape

In [24]:
#data.shape[1:]

In [25]:
#target_d

In [26]:
target_d.shape

torch.Size([6, 4, 2])

In [27]:
opt_conv = torch.optim.Adamax(convnet_sg.get_trainable_parameters(), lr=1e-3, betas=[0., .95])

# Their data_d is T, batchsize, pol, x, y (16, 4, 1, 28, 28)
# Mine is batchsize, T, pol, x,y

for e in range(epochs):        
    error = []
    accuracy=[]
    for item in tqdm(iter(batched_trainloader), desc=f"Epoch {e}"):
        data = item['data']
        label = item['target'] #.transpose(0, 1)

        #if label.shape[0] != 4:
            #print(f"last entry")
            #break
        
        convnet_sg.train()
        loss_hist = 0
        data_d = data.to(device)
        label_d = label.to(device)
        #print(len(convnet_sg))
        convnet_sg.init(data_d, burnin=3)
        readout = 0

        #print(label_d.shape)
        #print(label_d)

        data_d = data_d.transpose(0, 1)
        label_d = label_d.transpose(0, 1)
        
        #print(label_d.shape)
        #print(label_d)
        
        for n in range(num_bins_per_frame):
            # CORRECT: Data shape: torch.Size([4, 2, 480, 640]), overall: torch.Size([8, 4, 2, 480, 640])
            #print(f"Data shape: {data_d[n].shape}, overall: {data_d.shape}")
            #print(data_d[n])
            
            st, rt, ut = convnet_sg.forward(data_d[n])
            #print("Readout")
            #print(len(rt), rt[0].shape, rt[1].shape, rt[2].shape)
            #print(rt[-1].shape)

            # CORRECT: Label shape: torch.Size([4, 2]), overall: torch.Size([8, 4, 2])
            #print(f"Label shape: {label_d[n].shape}, overall: {label_d.shape}")
            #print(label_d[n].shape, label_d[n])
            
            #print(label_d)
            #break
            loss_tv = decolle_loss(rt, st, label_d[n])
            
            loss_tv.backward()
            opt_conv.step()
            opt_conv.zero_grad()
            loss_hist += loss_tv
            readout += rt[-1]

        #print(f"Readout: {readout}")
        #error += (readout.argmax(axis=1)!=label_d.argmax(axis=1)).float()
        #accuracy+=(readout.argmax(axis=1)==label_d.argmax(axis=1)).float()

        # SEEM CORRECT
        argmaxed_readout = readout.argmax(axis=1)
        #print("argmaxeds")
        #print(argmaxed_readout)
        #print(label_d)
        #print(label_d[-1])
        #print(label_d.argmax(axis=1))
        argmaxed_label = label_d[-1].argmax(axis=1)
        #print(argmaxed_label)
        #break
        #print("argmaxed readout")
        #print(argmaxed_readout)
        #print("argmaxed label")
        #print(argmaxed_label)
        error += (argmaxed_readout!=argmaxed_label).float()
        accuracy+=(argmaxed_readout==argmaxed_label).float()
        #print(f"Accuracy: {accuracy}")
        #break
        
    print('Training Error', torch.mean(torch.Tensor(error)).data)
    print('Training accuracy', torch.mean(torch.Tensor(accuracy)).data)     
    print('Epoch', e, 'Loss', loss_hist.data)
    PATH = './EVIMO_class_cnn.pth'  
    torch.save(convnet_sg.state_dict(), PATH)

Epoch 0:   0%|          | 0/80 [00:00<?, ?it/s]

Training Error tensor(0.4969)
Training accuracy tensor(0.5031)
Epoch 0 Loss tensor(4.5921, device='cuda:0')


Epoch 1:   0%|          | 0/80 [00:00<?, ?it/s]

Training Error tensor(0.4969)
Training accuracy tensor(0.5031)
Epoch 1 Loss tensor(4.5035, device='cuda:0')


Epoch 2:   0%|          | 0/80 [00:00<?, ?it/s]

...

Training Error tensor(0.4969)
Training accuracy tensor(0.5031)
Epoch 17 Loss tensor(4.4084, device='cuda:0')


Epoch 18:   0%|          | 0/80 [00:00<?, ?it/s]

Training Error tensor(0.4969)
Training accuracy tensor(0.5031)
Epoch 18 Loss tensor(4.3916, device='cuda:0')


Epoch 19:   0%|          | 0/80 [00:00<?, ?it/s]

Training Error tensor(0.4969)
Training accuracy tensor(0.5031)
Epoch 19 Loss tensor(3.8889, device='cuda:0')


In [28]:
type(loss_tv)

torch.Tensor

In [29]:
error = []
accuracy=[]
y_pred = []
y_true = []
#convnet_sg.requires_init = True
for item in iter(batched_trainloader):
    #print("Item")
    #print(item)
    data = item['data']
    label = item['target']
    if label.shape[0] != 4:
        #print(f"last entry: {label}")
        break

    #print("earliest label")
    #print(label)
    #print(label.shape)

    
    loss_hist = 0
    data_d = data.to(device)
    label_d = label.to(device)
    print(f"data_d shape: {data_d.shape}")
    print(f"data_d_tranpose shape: {data_d.transpose(0,1).shape}")
    print(f"Len convnet: {len(convnet_sg)}")
    print(f"data init shape: {data_d.transpose(0,1)[:, 0, :, :].shape}")
    print(f"Test data shape: {data_d[0].shape}")
    
    convnet_sg.init_evimo(data_d)
    readout = 0
    with torch.no_grad():
        #for i in range(0, len(data_d)):
        #d = data_d[i:i+1]
        
        #print(d.shape)
        
        st, rt, ut = convnet_sg.forward(data_d)

        
        
        print(f"Label shape: {label_d.shape}")
        print("Readout")
        print(rt)
        print(rt[0].shape)

        #print("labels")
        #print(label_d)
        #print(label_d.shape)
        
        
        loss_tv = decolle_loss(rt, st, label_d)
        
        
        loss_hist += loss_tv
        readout += rt[-1]
        print(f"Readout: {readout}")
        output = (readout.argmax(axis=1)).data.cpu().numpy()
        print(f"Output: {output}")
        y_pred.extend(output)
        #labels = label_d.argmax(axis=1)
        print(f"Labels: {label_d}")
        labels = (label_d.argmax(axis=1)).data.cpu().numpy()
        print(f"Labels: {labels}")
        y_true.extend(labels)
        accuracy+=(readout.argmax(axis=1)==label_d.argmax(axis=1)).float()
        error += (readout.argmax(axis=1)!=label_d.argmax(axis=1)).float()
        break
        
print('Testing Error', torch.mean(torch.Tensor(error)).data)
print('Testing accuracy', torch.mean(torch.Tensor(accuracy)).data)

Testing Error tensor(nan)
Testing accuracy tensor(nan)


In [30]:
print(y_true)
print(y_pred)

[]
[]


In [31]:
"""
convnet_sg = lenet_decolle_model.LenetDECOLLE( out_channels=2,
                    Nhid=[16,32], #Number of convolution channels
                    Mhid=[64],
                    kernel_size=[7],
                    pool_size=[2,2],
                    input_shape=[2, 480, 640],  # data.shape[1:],
                    alpha=[.95],
                    alpharp=[.65],
                    beta=[.92],
                    num_conv_layers=2,
                    num_mlp_layers=1,
                    lc_ampl=.5).to(device)

"""

'\nconvnet_sg = lenet_decolle_model.LenetDECOLLE( out_channels=2,\n                    Nhid=[16,32], #Number of convolution channels\n                    Mhid=[64],\n                    kernel_size=[7],\n                    pool_size=[2,2],\n                    input_shape=[2, 480, 640],  # data.shape[1:],\n                    alpha=[.95],\n                    alpharp=[.65],\n                    beta=[.92],\n                    num_conv_layers=2,\n                    num_mlp_layers=1,\n                    lc_ampl=.5).to(device)\n\n'

In [32]:
net = lenet_decolle_model.LenetDECOLLE(Nhid=[16,32],Mhid=[64],out_channels=2, pool_size=[2,2], num_conv_layers=2, num_mlp_layers=1, lc_ampl=.5, input_shape=[2, 480, 640])
d = torch.zeros([1, 2, 480, 640])
print(d.shape)
st, rt, ut = net.forward(d)
print(rt[0].shape)

LAYER SIZE: 1228800
STDV: 0.00045105489780439506
LAYER SIZE: 614400
STDV: 0.000637887953849786
LAYER SIZE: 64
STDV: 0.0625
torch.Size([1, 2, 480, 640])
torch.Size([1, 2])


In [33]:
print(rt)

[tensor([[0.0234, 0.1313]], grad_fn=<AddmmBackward0>), tensor([[0.1324, 0.1237]], grad_fn=<AddmmBackward0>), tensor([[ 0.1712, -0.0014]], grad_fn=<AddmmBackward0>)]


In [34]:
target = torch.zeros([1, 2])
target

tensor([[0., 0.]])

In [35]:
loss_tv = decolle_loss(rt, st, target)